In [1]:
import pandas as pd
import numpy as np
import csv
import os 
from  tqdm import tqdm
import librosa
from sklearn.preprocessing import LabelEncoder
import tensorflow  as tf


ModuleNotFoundError: No module named 'librosa'

Creating a metadata File

In [2]:
with open("audio_metadata.csv", 'w') as file:
  writer = csv.writer(file)   
  writer.writerow(["0", "1","2"])

Creating Columns for the metadata File

In [3]:
df = pd.read_csv('audio_metadata.csv', header=None)
df.rename(columns={0: 'audio_name', 1: 'audio_class',2: "audio_class_name"}, inplace=True)

In [6]:
df = pd.read_csv('audio_metadata.csv', header=None)

path="D:/Projects/MoodMate/speech_dataset"
count=0
for emotion in os.listdir(path):
    count+=1
    for audio in os.listdir(path + "/" + emotion):
        new_row = {'audio_name': audio, 'audio_class': count, 'audio_class_name': emotion}
        df = df.append(new_row, ignore_index=True)
        
df=df.drop([0,1,2],axis=1)
df.head()


df=df.iloc[1:]
df.to_csv("audio_metadata.csv")
df.head()

,audio_class,audio_class_name,audio_name
1,1.0,angry,angry_1.wav
2,1.0,angry,angry_10.wav
3,1.0,angry,angry_100.wav
4,1.0,angry,angry_1000.wav
5,1.0,angry,angry_1001.wav


In [7]:
metadata=pd.read_csv("audio_metadata.csv")
metadata.head()

,Unnamed: 0,audio_class,audio_class_name,audio_name
0,1,1.0,angry,angry_1.wav
1,2,1.0,angry,angry_10.wav
2,3,1.0,angry,angry_100.wav
3,4,1.0,angry,angry_1000.wav
4,5,1.0,angry,angry_1001.wav


In [8]:
def features_extractor(file_name):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    return mfccs_scaled_features

In [9]:
signal,sr = librosa.load("sudi.wav")
mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=40)
print(mfccs.shape)  

(40, 94)


In [10]:
extracted_features=[]

dataset_path="D:/Projects/MoodMate/speech_dataset"
for index_num,row in tqdm(metadata.iterrows()):
    file_name=dataset_path + "/" + row["audio_class_name"] + "/" + row["audio_name"]
    final_class_labels=row["audio_class_name"]
    data=features_extractor(file_name)
    extracted_features.append([data,final_class_labels])
    

20240it [09:54, 34.07it/s]


In [11]:
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head()

,feature,class
0,"[-362.86337, 31.40439, 0.7086624, 21.240759, -...",angry
1,"[-337.9935, 22.65596, -9.750982, 16.62628, -18...",angry
2,"[-344.77017, 14.45916, -7.1472893, 16.655972, ...",angry
3,"[-565.93616, 59.726994, -15.802911, 9.832316, ...",angry
4,"[-386.13574, 9.975171, -1.291482, 0.15957417, ...",angry


In [12]:
### Split the dataset into independent and dependent dataset
X=np.array(extracted_features_df['feature'].tolist())
Y=np.array(extracted_features_df['class'].tolist())
Y=np.array(pd.get_dummies(Y))

from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=0)

Y_test.shape

(4048, 7)

In [13]:
import tensorflow as tf
print(tf.__version__)

2.11.0


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [15]:
### No of classes
num_labels=Y.shape[1]
num_labels

7

In [16]:
model=Sequential()
###first layer
model.add(Dense(200,input_shape=(40,)))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))
###second layer
model.add(Dense(100))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))
###third layer
model.add(Dense(50))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))

###final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [17]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 200)               8200      
                                                                 
 activation (Activation)     (None, 200)               0         
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               20100     
                                                                 
 activation_1 (Activation)   (None, 100)               0         
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 50)                5

In [18]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [19]:
## Trianing my model
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

num_epochs = 150
num_batch_size = 64

checkpointer = ModelCheckpoint(filepath='saved_models/audio_classification.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, Y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, Y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/150
245/253 [============================>.] - ETA: 0s - loss: 1.9164 - accuracy: 0.2064
Epoch 1: val_loss improved from inf to 1.54002, saving model to saved_models\audio_classification.hdf5
253/253 [==============================] - 2s 3ms/step - loss: 1.9083 - accuracy: 0.2084 - val_loss: 1.5400 - val_accuracy: 0.4002
Epoch 2/150
245/253 [============================>.] - ETA: 0s - loss: 1.4883 - accuracy: 0.4094
Epoch 2: val_loss improved from 1.54002 to 1.12499, saving model to saved_models\audio_classification.hdf5
253/253 [==============================] - 1s 3ms/step - loss: 1.4843 - accuracy: 0.4120 - val_loss: 1.1250 - val_accuracy: 0.6635
Epoch 3/150
245/253 [============================>.] - ETA: 0s - loss: 1.1941 - accuracy: 0.5652
Epoch 3: val_loss improved from 1.12499 to 0.89076, saving model to saved_models\audio_classification.hdf5
253/253 [==============================] - 1s 3ms/step - loss: 1.1909 - accuracy: 0.5665 - val_loss: 0.8908 - val_accuracy: 0.7179

In [20]:
test_accuracy=model.evaluate(X_test,Y_test,verbose=0)
print(test_accuracy[1])

0.8777173757553101


In [21]:
0.866847813129425

0.866847813129425

In [ ]:
import pickle

# Assuming you have a trained model object named 'model'
# Save the model to a file using pickle
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [24]:
import h5py
from tensorflow import keras

# Assuming you have a trained Keras model object named 'model'
# Save the model to an HDF5 file
model.save('model.h5')

In [11]:
model = tf.keras.models.load_model('model.h5')
test_audio=features_extractor("D:/Projects/MoodMate/test_copy/anger.wav")
predictions = model.predict(test_audio)
print(predictions)


ValueError: in user code:

    File "c:\Users\sudha\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\engine\training.py", line 2137, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\sudha\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\engine\training.py", line 2123, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\sudha\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\engine\training.py", line 2111, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\sudha\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\engine\training.py", line 2079, in predict_step
        return self(x, training=False)
    File "c:\Users\sudha\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\sudh